# Merging Work Tasks to General Work Activities

The purpose of this notebook is to merge the original O\*NET data into one comprehesive CSV that will contain work tasks and their associated general work activities (GWA). The work tasks will be the input data into a machine learning model used to predict GWA, which act as labels.

**Last Updated**: Wednesday July 24, 2019
<br> **Author**: Rebecca Hu

<i> Note</i>: All original O\*NET data text files should be located in a folder labeled "original_onet_data". If that folder is no longer available, the data can be re-downloaded [here](https://www.onetcenter.org/database.html#act) (click the <i> Individual Files </i> Tab, then look under <i>Work Activities</i>). The files used include <i>DWA Reference.txt</i>, <i>Task Statements.txt</i>, <i>Tasks to DWAs.txt</i>, and <i>Work Activities.txt</i>.

In [1]:
# import necessary packages
import pandas as pd
import numpy as np
import re

Let's take a look at each of the files we grabbed from the O\*NET website. We're looking for a way to merge tasks with their associated GWA.

In [2]:
tasks = pd.read_csv('original_onet_data/Task Statements.txt', sep = '\t')

In [3]:
print(tasks.shape[0], 'rows')
tasks.head()

19636 rows


,O*NET-SOC Code,Task ID,Task,Task Type,Incumbents Responding,Date,Domain Source
0,11-1011.00,8823,Direct or coordinate an organization's financi...,Core,87.0,07/2014,Incumbent
1,11-1011.00,8831,Appoint department heads or managers and assig...,Core,87.0,07/2014,Incumbent
2,11-1011.00,8825,Analyze operations to evaluate performance of ...,Core,87.0,07/2014,Incumbent
3,11-1011.00,8826,"Direct, plan, or implement policies, objective...",Core,87.0,07/2014,Incumbent
4,11-1011.00,8827,"Prepare budgets for approval, including those ...",Core,87.0,07/2014,Incumbent


<i>Task Statements.txt</i> contains the text of each individual task which we will definitely need, as well as the tasks' associated SOC Codes and Task IDs which could be useful for merging multiple pd.DataFrame objects together. Notice that the SOC codes are not unique to the tasks, but the Task IDs are.

In [4]:
task_to_dwa = pd.read_csv('original_onet_data/Tasks to DWAs.txt', sep = '\t')

In [5]:
print(task_to_dwa.shape[0], 'rows')
task_to_dwa.head()

22936 rows


,O*NET-SOC Code,Task ID,DWA ID,Date,Domain Source
0,11-1011.00,20461,4.A.2.a.4.I09.D03,07/2014,Analyst
1,11-1011.00,20461,4.A.4.b.6.I08.D04,07/2014,Analyst
2,11-1011.00,8823,4.A.4.b.4.I09.D02,03/2014,Analyst
3,11-1011.00,8824,4.A.4.a.2.I03.D14,03/2014,Analyst
4,11-1011.00,8825,4.A.2.a.4.I07.D09,03/2014,Analyst


<i>Tasks to DWAs.txt</i> also contains SOC Codes and Task IDs, but it also contains information about the Detailed Work Activity (DWA) associated with each task. We don't necessarily need any information about a task's DWA, but it could be useful for intermediate merging. Notice that in this table, Task IDs are not unique, so one task could have multiple DWAs associated with it.

In [6]:
gwa = pd.read_csv('original_onet_data/Work Activities.txt', sep = '\t')

In [7]:
print(gwa.shape[0], 'rows')
gwa.head()

79294 rows


,O*NET-SOC Code,Element ID,Element Name,Scale ID,Data Value,N,Standard Error,Lower CI Bound,Upper CI Bound,Recommend Suppress,Not Relevant,Date,Domain Source
0,11-1011.00,4.A.1.a.1,Getting Information,IM,4.72,35.0,0.13,4.46,4.98,N,NaN,07/2014,Incumbent
1,11-1011.00,4.A.1.a.1,Getting Information,LV,5.35,35.0,0.23,4.89,5.81,N,N,07/2014,Incumbent
2,11-1011.00,4.A.1.a.2,"Monitor Processes, Materials, or Surroundings",IM,3.68,35.0,0.17,3.33,4.03,N,NaN,07/2014,Incumbent
3,11-1011.00,4.A.1.a.2,"Monitor Processes, Materials, or Surroundings",LV,4.76,34.0,0.27,4.22,5.30,N,N,07/2014,Incumbent
4,11-1011.00,4.A.1.b.1,"Identifying Objects, Actions, and Events",IM,4.20,35.0,0.15,3.91,4.50,N,NaN,07/2014,Incumbent


<i>Work Activities.txt</i> contains the GWA we need to label the Task information with under the column header "Element Name". Each GWA is also associated with a unique ID under "Element ID". We should be able to use these IDs to merge the GWAs with the Tasks. (Also, it is unclear what each row in this file represents (?) but we can still utilize the information in provides on GWAs for our purposes)

In [8]:
dwa = pd.read_csv('original_onet_data/DWA Reference.txt', sep = '\t')

In [9]:
print(dwa.shape[0], 'rows')
dwa.head()

2070 rows


,Element ID,IWA ID,DWA ID,DWA Title
0,4.A.1.a.1,4.A.1.a.1.I01,4.A.1.a.1.I01.D01,Review art or design materials.
1,4.A.1.a.1,4.A.1.a.1.I01,4.A.1.a.1.I01.D02,Study details of musical compositions.
2,4.A.1.a.1,4.A.1.a.1.I01,4.A.1.a.1.I01.D03,Review production information to determine cos...
3,4.A.1.a.1,4.A.1.a.1.I01,4.A.1.a.1.I01.D04,Study scripts to determine project requirements.
4,4.A.1.a.1,4.A.1.a.1.I02,4.A.1.a.1.I02.D01,Read materials to determine needed actions.


<i>DWA Reference.txt</i> maps each DWA to its associated GWA ID (In this data, GWA are referred to as Elements). Notice that the data under the column headers "Element ID", "IWA ID", and "DWA ID" are all related. The DWA ID numbers are extensions of the IWA ID which are extensions of the Element ID (or GWA ID).

---

Now that we have an idea of the data we have, we can determine how to best isolate the data we're interested in using. We know we have to start with the pd.DataFrame, <i>tasks</i>, because it is the only object that contains the text of each indivdual work task. Then, we know that we'll need to combine the work tasks in the <i>tasks</i> object to the <i>gwa</i> pd.DataFrame in some way, as <i>gwa</i> is the only object with information about the GWAs. We can use the <i>dwa</i> and <i>task_to_dwa</i> pd.DataFrames as intermediaries to help us link together the <i>tasks</i> and <i>gwa</i> pd.DataFrames.

<b>Strategy</b>: Merge the <i>tasks</i> and <i>task_to_dwa</i> pd.DataFrames together using the column "Task ID". Then, we'll have the DWA ID associated with each task. We previously noticed that the GWA ID can be derrived from the IWA and DWA IDs, so now we have a way to combine the information for the tasks with their GWA IDs. Lastly, we can use the GWA IDs to connect the tasks to the text of the GWAs using the <i>gwa</i> object. (Note: We ended up not needing the <i>dwa</i> pd.DataFrame. Also, we could have saved a few steps by just connecting the Task IDs with their GWA IDs, but using the human-readable text for the tasks and GWAs will make it easier for us to conduct prelimanary exploratory data analysis later)

In [10]:
#create dictionary of {gwa id: gwa descriptions}
gwa_dict = pd.Series(gwa['Element Name'].values, index=gwa['Element ID']).to_dict()
print(len(gwa_dict))
gwa_dict

41


{'4.A.1.a.1': 'Getting Information',
 '4.A.1.a.2': 'Monitor Processes, Materials, or Surroundings',
 '4.A.1.b.1': 'Identifying Objects, Actions, and Events',
 '4.A.1.b.2': 'Inspecting Equipment, Structures, or Material',
 '4.A.1.b.3': 'Estimating the Quantifiable Characteristics of Products, Events, or Information',
 '4.A.2.a.1': 'Judging the Qualities of Things, Services, or People',
 '4.A.2.a.2': 'Processing Information',
 '4.A.2.a.3': 'Evaluating Information to Determine Compliance with Standards',
 '4.A.2.a.4': 'Analyzing Data or Information',
 '4.A.2.b.1': 'Making Decisions and Solving Problems',
 '4.A.2.b.2': 'Thinking Creatively',
 '4.A.2.b.3': 'Updating and Using Relevant Knowledge',
 '4.A.2.b.4': 'Developing Objectives and Strategies',
 '4.A.2.b.5': 'Scheduling Work and Activities',
 '4.A.2.b.6': 'Organizing, Planning, and Prioritizing Work',
 '4.A.3.a.1': 'Performing General Physical Activities',
 '4.A.3.a.2': 'Handling and Moving Objects',
 '4.A.3.a.3': 'Controlling Machines

In [11]:
#create dictionary of {task ids: task descriptions}
task_dict = pd.Series(tasks['Task'].values, index=tasks['Task ID']).to_dict()
task_dict

{8823: "Direct or coordinate an organization's financial or budget activities to fund operations, maximize investments, or increase efficiency.",
 8831: 'Appoint department heads or managers and assign or delegate responsibilities to them.',
 8825: 'Analyze operations to evaluate performance of a company or its staff in meeting objectives or to determine areas of potential cost reduction, program improvement, or policy change.',
 8826: 'Direct, plan, or implement policies, objectives, or activities of organizations or businesses to ensure continuing operations, to maximize returns on investments, or to increase productivity.',
 8827: 'Prepare budgets for approval, including those for funding or implementation of programs.',
 8824: 'Confer with board members, organization officials, or staff members to discuss issues, coordinate activities, or resolve problems.',
 8836: 'Implement corrective action plans to solve organizational or departmental problems.',
 8832: 'Direct human resources 

In [12]:
# This cell takes ~2 min to run

# Take only the columns "Task ID" and "DWA ID" from the task_to_dwa pd.DataFrame then replace the Task IDs with the task text 
task_to_gwa = task_to_dwa[['Task ID', 'DWA ID']].replace(task_dict)
# then parse the DWA ID into the GWA IDs
task_to_gwa['DWA ID'] = task_to_gwa.apply(lambda row: row['DWA ID'][:9], axis = 1)
# then replace the GWA IDs with the GWA text    [.replace(gwa_dict)]
task_to_gwa = task_to_gwa.replace(gwa_dict)
task_to_gwa.head()

,Task ID,DWA ID
0,"Review and analyze legislation, laws, or publi...",Analyzing Data or Information
1,"Review and analyze legislation, laws, or publi...",Provide Consultation and Advice to Others
2,Direct or coordinate an organization's financi...,"Guiding, Directing, and Motivating Subordinates"
3,"Confer with board members, organization offici...","Communicating with Supervisors, Peers, or Subo..."
4,Analyze operations to evaluate performance of ...,Analyzing Data or Information


In [13]:
task_to_gwa.columns = ['Task', 'GWA'] # rename the column headers
task_to_gwa = task_to_gwa.drop_duplicates() # drop identical rows
task_to_gwa.to_csv('onet_tasks_gwas.csv', sep = ',', index = False) # write to csv